In [1]:
# Code to get landmarks from mediapipe and then create a bounding box around dip joints to segment them 
# for further analysis

# Use distance transform to further correct the landmarks from mediapipe

# See if you can get tip indices from drawing a convex hull around hand??

# NOTE: Current code works on already segmented masks - this preprocessing step is required!

In [2]:
# import required libraries

import cv2 as cv # opencv
import numpy as np # for numerical calculations 
import synapseclient # synapse login etc
import synapseutils # to download files using folder structure
import pandas as pd # data frames
from matplotlib import pyplot as plt # plotting
import mediapipe as mp # for detecting hand landmarks
import seaborn as sns # for violin plot
import os # for listing files in a folder
import timeit # to track program running time

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
# login into Synapse
syn = synapseclient.login()

## hand landmark detection using mediapipe
hands = mp_hands.Hands(static_image_mode=True,
                       max_num_hands=1, # use one hand at a time 
                       min_detection_confidence=0.5)

Welcome, Meghasyam Tummalacherla!




UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.5.0) is available. Your version (2.4.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.5.0 release notes

https://python-docs.synapse.org/build/html/news.html

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def getTIPLandmarks(multi_hand_landmarks, img_shape):
    # input is results.multi_hand_landmarks
    # will focus on all fingers except thumb - i.e index, middle, ring and pinky
    for hand_landmarks in multi_hand_landmarks:
        index_finger_tip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y* img_shape[0])]
        middle_finger_tip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y* img_shape[0])]
        ring_finger_tip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y* img_shape[0])]
        pinky_tip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y* img_shape[0])]

    tips_dict = {'index': index_finger_tip, 'middle': middle_finger_tip, 'ring': ring_finger_tip, 'pinky': pinky_tip}
    return(tips_dict)            

def getDIPLandmarks(multi_hand_landmarks, img_shape):
    # input is results.multi_hand_landmarks
    # will focus on all fingers except thumb - i.e index, middle, ring and pinky
    for hand_landmarks in multi_hand_landmarks:
        index_finger_dip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y* img_shape[0])]
        middle_finger_dip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y* img_shape[0])]
        ring_finger_dip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP].y* img_shape[0])]
        pinky_dip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP].y* img_shape[0])]

    dips_dict = {'index': index_finger_dip, 'middle': middle_finger_dip, 'ring': ring_finger_dip, 'pinky': pinky_dip}
    return(dips_dict)            

def getPIPLandmarks(multi_hand_landmarks, img_shape):
    # input is results.multi_hand_landmarks
    # will focus on all fingers except thumb - i.e index, middle, ring and pinky
    for hand_landmarks in multi_hand_landmarks:
        index_finger_pip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y* img_shape[0])]
        middle_finger_pip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y* img_shape[0])]
        ring_finger_pip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y* img_shape[0])]
        pinky_pip = [round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].x* img_shape[1]), round(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y* img_shape[0])]

    dips_dict = {'index': index_finger_pip, 'middle': middle_finger_pip, 'ring': ring_finger_pip, 'pinky': pinky_pip}
    return(dips_dict) 

def getHalfPoints(landmarks_1, landmarks_2):
    # input is two landmarks dictionaries, eg., tips_landmarks and dips_landmarks
    index_landmarks = np.round(np.add(landmarks_1['index'], landmarks_2['index'])/2)
    middle_landmarks = np.round(np.add(landmarks_1['middle'], landmarks_2['middle'])/2)
    ring_landmarks = np.round(np.add(landmarks_1['ring'], landmarks_2['ring'])/2)
    pinky_landmarks = np.round(np.add(landmarks_1['pinky'], landmarks_2['pinky'])/2)
    
    return_dict = {'index': index_landmarks, 'middle': middle_landmarks, 'ring': ring_landmarks, 'pinky': pinky_landmarks}
    return(return_dict)
    

def getTipsFromHull(tips_landmarks, hull):
    # input is TIPLandmarks and hull points
    # For each landmark we find the nearest hull point, i.e matching the hull points
    # with the tips, so that we can associate which sections of the contour belong 
    # to which finger
    
    # index finger
    index_dist = list(map(lambda x: np.linalg.norm(x-tips_landmarks['index']), hull[0]))
    index_hull_point = hull[0][index_dist.index(min(index_dist))][0]
    
    # middle finger
    middle_dist = list(map(lambda x: np.linalg.norm(x-tips_landmarks['middle']), hull[0]))
    middle_hull_point = hull[0][middle_dist.index(min(middle_dist))][0]

    # ring finger
    ring_dist = list(map(lambda x: np.linalg.norm(x-tips_landmarks['ring']), hull[0]))
    ring_hull_point = hull[0][ring_dist.index(min(ring_dist))][0]

    # pinky 
    pinky_dist = list(map(lambda x: np.linalg.norm(x-tips_landmarks['pinky']), hull[0]))
    pinky_hull_point = hull[0][pinky_dist.index(min(pinky_dist))][0]

    tips_hulls_dict = {'index': index_hull_point, 'middle': middle_hull_point, 'ring': ring_hull_point, 'pinky': pinky_hull_point}
    return(tips_hulls_dict)            

def getClosestPixelInHull(pixel_in, contour_section):
    # Given a pixel_in and contour_section, output the pixel in the contour_section that is closese to pixel_in
    index_dist = list(map(lambda x: np.linalg.norm(x-pixel_in), contour_section[0]))
    index_point = contour_section[0][index_dist.index(min(index_dist))][0]
    return(index_point)


def locatePixelInList(pixel,input_list):
    # Given a contour find the index of the input pixel (mostly one from the convex hull)
    
    temp_list = list([(input_list[0][x][0] == pixel).all() for x in range(len(input_list[0]))])
    # gives a list of true/false 
    pixel_index = temp_list.index(max(temp_list))
    # pick the true
    
    return(pixel_index)

In [5]:
def getHand(img, hand= 'left'):
    """
    Split the given two-hand image into a single hand image.
        
    :param img:   input RGB image
    :param hand:  'left' - left half of the picture 
                  'right' - right half of the image
        
    """
    rows, cols, channels = img.shape
    new_cols = round(cols/2)

    
    if hand == 'left':
        img_cropped = img[:, 0:new_cols-1,:]
    elif hand == 'right':
        img_cropped = img[:, new_cols:cols-1,:]
    else:
        print('Returning input image')
        img_cropped = img
    
    return img_cropped
    

def getBinaryImage(im, min_foreground_pixel = 10, max_foreground_pixel = 255):
    # Thresholds the given image(segmented mask with black background) to give a black-white binary image
    # with the background being white pixels and foreground(hand) being white pixels
    
    ### Convert to Gray Scale
    imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)

    ### Binary Threshold
    ret, thresh = cv.threshold(imgray, min_foreground_pixel, max_foreground_pixel, cv.THRESH_BINARY)
    
    return(thresh)

def getContoursAndHull(black_white_im):
    # Given a binary(black and white image with segmented hand mask) give output of contours, hierarchy
    # and convex hull points

    # returns only the largest contour as this is the one we want for the hand!
    
    ### Find contours
    contours, hierarchy = cv.findContours(black_white_im, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    
    ### contour lengths
    contour_lengths = [len(cc) for cc in contours]
    largest_contour_index = contour_lengths.index(max(contour_lengths))
    
    # subset contours and hierarchy to the largest contour
    contours = [contours[largest_contour_index]]
    hierarchy = np.array([[hierarchy[0][largest_contour_index]]])
    
    ### Convex Hull
    # create hull array for convex hull points
    hull = []
    # calculate points for each contour
    for i in range(len(contours)):
        # creating convex hull object for each contour
        hull.append(cv.convexHull(contours[i], False))
    
    return({'contours':contours, 'hierarchy':hierarchy, 'hull': hull})

def drawConvexHull(black_white_im, contours, hierarchy, hull, draw_Contour = True):
    # given a black white image, contours and hull, output a image with contours and hull drawn
    
    # create an empty black image
#     drawing = np.zeros((black_white_im.shape[0], black_white_im.shape[1], 3), np.uint8)
    
    drawing = black_white_im

    # draw contours and hull points
    color_contours = (0, 255, 0) # green - color for contours
    color = (255, 0, 0) # blue - color for convex hull
    
    # draw ith contour
    if draw_Contour:
        cv.drawContours(drawing, contours, -1, color_contours, 1, 8, hierarchy)
    
    # draw ith convex hull object
    cv.drawContours(drawing, hull, -1, color, 1, 8)
    
    return(drawing)



In [6]:
def cropImageFromContour(img, cnt):
    # Crop the image(img) based on the input of a closed contour(cnt), set of points
    # adapted from https://www.life2coding.com/cropping-polygon-or-non-rectangular-region-from-image-using-opencv-python/
    mask = np.zeros(img.shape[0:2], dtype=np.uint8)
    
    # draw the contours on the mask
    cv.drawContours(mask, [cnt], -1, (255, 255, 255), -1, cv.LINE_AA)
    
    res = cv.bitwise_and(img,img,mask = mask)
    rect = cv.boundingRect(cnt) # returns (x,y,w,h) of the rect
    cropped = res[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]
    
#     ## To get white background cropped image
#     ## create the white background of the same size of original image
#     wbg = np.ones_like(img, np.uint8)*255
#     cv.bitwise_not(wbg,wbg, mask=mask)
    
#     # overlap the resulted cropped image on the white background
#     dst = wbg+res
#     dst_cropped = dst[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]

    return(cropped)

def cropWarpedRect(img, rect):
    #### BEST
    
    # from https://stackoverflow.com/questions/11627362/how-to-straighten-a-rotated-rectangle-area-of-an-image-using-opencv-in-python/48553593#48553593
    # Get center, size, and angle from rect
    center, size, theta = rect
    
    box = cv.boxPoints(rect)
    box = np.int0(box)
    width = int(rect[1][0])
    height = int(rect[1][1])

    src_pts = box.astype("float32")
    dst_pts = np.array([[0, height-1],
                        [0, 0],
                        [width-1, 0],
                        [width-1, height-1]], dtype="float32")
    M = cv.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv.warpPerspective(img, M, (width, height))
    if (theta > 45):
        warped = cv.rotate(warped, cv.ROTATE_90_CLOCKWISE)

    return(warped)

In [7]:
def getContourSubsectionPercentage(contour_in, pixel_in, percent = 5):
    # get subsection of contour that is 5%(default) left and right to the input pixel
    input_pixel_location = locatePixelInList(pixel_in, contour_in)  
    nPixel_contour = len(contour_in[0])
    
    section_left = int(max(0, input_pixel_location - round(nPixel_contour*percent/100)))
    section_right = int(min(nPixel_contour-1, input_pixel_location + round(nPixel_contour*percent/100)))  
    
    print(section_left)
    print(section_right)
    
    subContour = [np.array(contour_in[0][section_left:section_right])]
    
    return(subContour)

In [8]:
def getContourSubsection(contour_in, pixel_on_contour, ref_pixel):
    # Given a pixel on the contour, split the contour into two sections left of the pixel
    # and right of the pixel.
    # Then find the closest points in both sections (left and right) to the ref_pixel, and 
    # then give subset the contour between these two points
    # The idea is when we give a contour of the hand, and a point on it corresponding to the
    # finger tip, we get two sections of the contour, to the left of the finger enf then right
    # We then find the points closest to these 
    
    input_pixel_location = locatePixelInList(pixel_on_contour, contour_in)  
    nPixel_contour = len(contour_in[0])
    
    # roll/shift the array so that input_pixel is the middle of array
    contour_rolled = [np.array(np.roll(contour_in[0],2*(round(nPixel_contour/2)-input_pixel_location)))]
    
    section_left = [np.array(contour_rolled[0][0:round(nPixel_contour/2)])]
    section_right = [np.array(contour_rolled[0][round(nPixel_contour/2):nPixel_contour])]
    
    closest_pixel_left = getClosestPixelInHull(ref_pixel, section_left)
    closest_pixel_right = getClosestPixelInHull(ref_pixel, section_right)
    
    closest_pixel_left_location = locatePixelInList(closest_pixel_left, contour_rolled)
    closest_pixel_right_location = locatePixelInList(closest_pixel_right, contour_rolled)
    
    subContour = [np.array(contour_rolled[0][(closest_pixel_left_location-1):closest_pixel_right_location])]
    subContour = [np.array(np.roll(subContour[0],-2*(round(nPixel_contour/2)-input_pixel_location)))]
    
#     subContour = [np.array([[pixel_on_contour], [closest_pixel_left], [closest_pixel_right]])]
#     return({'left': closest_pixel_left, 'right': closest_pixel_right})
    return(subContour)
    

In [9]:
def getContourSubsectionJoint(contour_in, pixel_on_contour, ref_pixel_1, ref_pixel_2):
    # Given a pixel on the contour, split the contour into two sections left of the pixel
    # and right of the pixel.
    # Then find the closest points in both sections (left and right) to the ref_pixels, and 
    # then give subset the contour between these two points
    # ref_pixel_2 is farther from tip hull than ref_pixel_1
    # The idea is when we give a contour of the hand, and a point on it corresponding to the
    # finger tip, we get two sections of the contour, to the left of the finger enf then right
    # We then find the points closest to these 
    
    input_pixel_location = locatePixelInList(pixel_on_contour, contour_in)  
    nPixel_contour = len(contour_in[0])
    
    # roll/shift the array so that input_pixel is the middle of array
    contour_rolled = [np.array(np.roll(contour_in[0],2*(round(nPixel_contour/2)-input_pixel_location)))]
    
    section_left = [np.array(contour_rolled[0][0:round(nPixel_contour/2)])]
    section_right = [np.array(contour_rolled[0][round(nPixel_contour/2):nPixel_contour])]
    
    closest_pixel_left_1 = getClosestPixelInHull(ref_pixel_1, section_left)
    closest_pixel_right_1 = getClosestPixelInHull(ref_pixel_1, section_right)
    
    closest_pixel_left_1_location = locatePixelInList(closest_pixel_left_1, contour_rolled)
    closest_pixel_right_1_location = locatePixelInList(closest_pixel_right_1, contour_rolled)

    closest_pixel_left_2 = getClosestPixelInHull(ref_pixel_2, section_left)
    closest_pixel_right_2 = getClosestPixelInHull(ref_pixel_2, section_right)
    
    closest_pixel_left_2_location = locatePixelInList(closest_pixel_left_2, contour_rolled)
    closest_pixel_right_2_location = locatePixelInList(closest_pixel_right_2, contour_rolled)

    subContour_1 = [np.array(contour_rolled[0][(closest_pixel_left_1_location-1):closest_pixel_right_1_location])]
    subContour_1 = [np.array(np.roll(subContour_1[0],-2*(round(nPixel_contour/2)-input_pixel_location)))]
    
    subContour_2 = [np.array(contour_rolled[0][(closest_pixel_left_2_location-1):closest_pixel_right_2_location])]
    subContour_2 = [np.array(np.roll(subContour_2[0],-2*(round(nPixel_contour/2)-input_pixel_location)))]
   
    subContour = [np.array([np.array([x]) for x in ((set([(tuple(x[0])) for x in subContour_2[0]]))^(set([(tuple(x[0])) for x in subContour_1[0]])))])]
    
#     subContour = {'1': subContour_1, '2': subContour_2}
    
#     subContour = subContour_2
#     subContour = [np.array([[pixel_on_contour], [closest_pixel_left], [closest_pixel_right]])]
#     return({'left': closest_pixel_left, 'right': closest_pixel_right})
    return(subContour)
    

In [10]:
# In the working folder (..../), have the following folder structure
# ..../hand_segmentation_psorcast Jun 16 2021 08_51 Dan/
# ..../testHullsDIP
# ..../testHullsDIP/left
# ..../testHullsDIP/right
# ..../segmentedDIP
# ..../segmentedDIP/left
# ..../segmentedDIP/left_unrotated
# ..../segmentedDIP/right
# ..../segmentedDIP/right_unrotated

In [11]:
## Getting hands images from Synapse
hand_masks_synapse_id = 'syn25999658' # Folder containing all merged hand images and masks
img_all_directory = 'hand_segmentation_psorcast Jun 16 2021 08_51 Dan/' # Local path for storing images from Synapse
# Hand mask images form slack (check June 16 - Chat with Dan Webster, Aryton Tediarjo and Meghasyam Tummalacherla)
# Also in https://www.synapse.org/#!Synapse:syn25999658

# Download the curated hand image files from Synapse
# hands_all_files = synapseutils.syncFromSynapse(syn = syn, entity= hand_masks_synapse_id, path= img_all_directory)

all_files = os.listdir(img_all_directory)

fuse_files = list(filter(lambda x: 'fuse' in x, all_files))
orig_files = list(map(lambda x: x[:-11], fuse_files)) # remove ___fuse.png

target_directory = 'testHullsDIP' # directory with mediapipe results
target_dip_directory = 'segmentedDIP' # directory with segmented DIP joints

In [12]:
## Getting full - resolution hand images from synapse
full_res_images_synapse_id = 'syn25837496' # Folder containing full resolution images
img_full_res_directory = 'hand_images_full_res' # local path to download the files into

# Download the curated hand image files from Synapse
# hands_all_files = synapseutils.syncFromSynapse(syn = syn, entity= full_res_images_synapse_id, path= img_full_res_directory)

all_files = os.listdir(img_all_directory)

In [14]:
## Target Synapse directory for segmented dips
dips_syn_target_folder = 'syn26381177'

def customSynapseUpload(file_path, file_name):
    # provenance
    provenance_set = synapseclient.Activity(
        name = file_name,
        used = 'syn25999658')

    test_entity = synapseclient.File(
        file_path,
        parent=dips_syn_target_folder)

    test_entity = syn.store(test_entity, activity = provenance_set)

In [15]:
#### LEFT HAND

left_fail_index = 0
left_pass_index = 0
left_dip_rects = {} # save image name and rect dictionaries, i.e image vs dip bounding boxes

for index in range(len(fuse_files)):
    
    # update path for current image
    current_image_path = img_all_directory + '/' + fuse_files[index]
#     current_orig_path = img_all_directory + '/' + orig_files[index]
    current_orig_path = img_full_res_directory + '/' + orig_files[index] # take full res image in    
    
    print(current_image_path)
    
    # update target path for the contoured image for the current image
    current_left_target_path = target_directory + '/' + 'left/' + fuse_files[index]
    
    # update target path for segmented dips from the current image (ROTATED)
    current_left_dip_target_path = target_dip_directory + '/' + 'left'

    # update target path for segmented dips from the current image (UN-ROTATED)
    current_left_dip_target_path_unrotated = target_dip_directory + '/' + 'left_unrotated'

    # read images
    img = cv.imread(current_image_path)
    orig_img = cv.imread(current_orig_path)
    
    # Masks of left and right img (fuse files)
    left_img_fuse = getHand(img, 'left')
    
    # Actual photos
    left_orig_img = getHand(orig_img, 'left')
    
    # clones of actual images
    left_orig_img_clone = left_orig_img.copy()
    
    # resize image [for faster calculation, and mediapipe usually takes in small images of size
    # 300 x 300][https://github.com/google/mediapipe/blob/master/mediapipe/calculators/image/scale_image_calculator.cc]
    img_shape  = left_orig_img.shape
    resize_factor = round(300/max(img_shape),3) # resize max(length, width) to 300 pixels
    left_orig_img_resize = cv.resize(left_orig_img, None, fx = resize_factor , fy = resize_factor, interpolation = cv.INTER_AREA)
    
    color_contours = (0, 255, 0) # green - color for contours
        
    ### left hand
    bw_img_fuse = getBinaryImage(left_img_fuse)

    # resize the left hand mask to match that of the original image
    img_shape = left_orig_img.shape[0:2]
    bw_img = cv.resize(bw_img_fuse, (img_shape[1], img_shape[0]), interpolation = cv.INTER_AREA)
    
    ###
    contours_hull = getContoursAndHull(bw_img)

    ###
    dw_img = drawConvexHull(left_orig_img, contours_hull['contours'], contours_hull['hierarchy'], contours_hull['hull'])

    # apply mediapipe to get hand landmarks
    results = hands.process(cv.cvtColor(left_orig_img_resize, cv.COLOR_BGR2RGB))

    # to draw all landmarks from mediapipe
    if not results.multi_hand_landmarks:
        cv.imwrite(current_left_target_path, dw_img)
        left_dip_rects[orig_files[index]] = {}
        left_fail_index = left_fail_index + 1
    else:
        dw_img = drawConvexHull(left_orig_img, contours_hull['contours'], contours_hull['hierarchy'], contours_hull['hull'], draw_Contour=True)
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(dw_img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # get tip landmarks from results
        tips_landmarks = getTIPLandmarks(results.multi_hand_landmarks, left_orig_img.shape)
        
        # get DIP landmarks from results
        dips_landmarks = getDIPLandmarks(results.multi_hand_landmarks, left_orig_img.shape)
        
        # get PIP landmarks from results
        pips_landmarks = getPIPLandmarks(results.multi_hand_landmarks, left_orig_img.shape)
               
        ## get middle points b/2 dips_landmarks and pips_landmarks
        dip_pip_landmarks = getHalfPoints(dips_landmarks, pips_landmarks)
        
        # get points closest to tips from hull
        tips_hull = getTipsFromHull(tips_landmarks, contours_hull['hull'])
        
        ## get middle points b/w tips_hull and dips_landmarks
        tip_dip_landmarks = getHalfPoints(tips_landmarks, dips_landmarks)
 
        # get subcontours for each finger
        subContours = {}
        
        # get minimum bounding rectangle(min area) for each subContour
        subContourRects = {}
        subContourBoxes = {}
        
        for finger_key in tips_hull:
            subContours[finger_key] = getContourSubsectionJoint(contours_hull['contours'], tips_hull[finger_key],tip_dip_landmarks[finger_key], dip_pip_landmarks[finger_key])
#             subContours[finger_key] = getContourSubsectionPercentage(contours_hull['contours'], tips_hull[finger_key])
            rect = cv.minAreaRect(subContours[finger_key][0])
            box = cv.boxPoints(rect)
            box = np.int0(box)
            subContourRects[finger_key] = rect
            subContourBoxes[finger_key] = [box]
    
         # draw SubContours finger
        for finger_key in subContours:
            cv.drawContours(dw_img, subContours[finger_key], -1, (255,0,0), 1, 8, contours_hull['hierarchy'])
            cv.drawContours(dw_img, subContourBoxes[finger_key],-1,(0,0,255))
            
            # rotated via rects
            cropped_finger = cropWarpedRect(left_orig_img_clone, subContourRects[finger_key])
            curr_file_name = 'left_' + finger_key + '_' + fuse_files[index]
            curr_path = current_left_dip_target_path + '/' + curr_file_name
            if min(cropped_finger.shape) >0 :
                cv.imwrite(curr_path, cropped_finger)
                # upload this corrected cropped finger to synapse via customSynpaseUpload
                customSynapseUpload(curr_path, curr_file_name)
                
            # unrotated fingers
            cropped_finger_unrotated = cropImageFromContour(left_orig_img_clone, subContourBoxes[finger_key][0])
            curr_file_name = 'left_' + finger_key + '_' + fuse_files[index]
            curr_path = current_left_dip_target_path_unrotated + '/' + curr_file_name
            if min(cropped_finger_unrotated.shape) >0 :
                cv.imwrite(curr_path, cropped_finger_unrotated)
            
        cv.imwrite(current_left_target_path, dw_img)
        left_dip_rects[orig_files[index]] = subContourRects
        left_pass_index = left_pass_index + 1
    
print('left fail percent')
print(100*left_fail_index/(left_fail_index+left_pass_index))


hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_032_handImaging_2020-12-02_20.49.05.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_008_handImaging_2020-03-04_15.43.41.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE2_016_handImaging_2020-11-11_20.57.08.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_001_handImaging_2020-02-20_21.02.56.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_036_handImaging_2021-03-24_14.43.51.jpg___fuse.png

##################################################
 Uploading file to Synapse storage

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_003_handImaging_2020-02-21_15.36.43.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_018_handImaging_2020-09-02_17.45.39.jpg___fuse.png
hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE2_026_handImaging_2021-02-10_20.37.53.jpg___fuse.png
hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE2_019_handImaging_2020-12-02_18.47.31.jpg___fuse.png

###########


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE2_009_handImaging_2020-11-04_18.38.43.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_022_handImaging_2020-09-10_20.26.44.jpg___fuse.png

##################################################
 Uploading file to Synapse storage

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE2_021_handImaging_2020-12-09_21.17.48.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE2_018_handImaging_2020-11-12_16.48.00.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage

In [16]:
#### RIGHT HAND

right_fail_index = 0
right_pass_index = 0
right_dip_rects = {} # save image name and rect dictionaries, i.e image vs nail bounding boxes


for index in range(len(fuse_files)):
    
    # update path for current image
    current_image_path = img_all_directory + '/' + fuse_files[index]
#     current_orig_path = img_all_directory + '/' + orig_files[index]
    current_orig_path = img_full_res_directory + '/' + orig_files[index] # take full res image in    

    print(current_image_path)
    
    # update target path for the contoured image for the current image
    current_right_target_path = target_directory + '/' + 'right/' + fuse_files[index]
    
    # update target path for segmented dips from the current image (ROTATED)
    current_right_dip_target_path = target_dip_directory + '/' + 'right'

    # update target path for segmented dips from the current image (UN-ROTATED)
    current_right_dip_target_path_unrotated = target_dip_directory + '/' + 'right_unrotated'

    # read images
    img = cv.imread(current_image_path)
    orig_img = cv.imread(current_orig_path)
    
    # Masks of left and right img (fuse files)
    right_img_fuse = getHand(img, 'right')
    
    # Actual photos
    right_orig_img = getHand(orig_img, 'right')
    
    # clones of actual images
    right_orig_img_clone = right_orig_img.copy()
    
    # resize image [for faster calculation, and mediapipe usually takes in small images of size
    # 300 x 300][https://github.com/google/mediapipe/blob/master/mediapipe/calculators/image/scale_image_calculator.cc]
    img_shape  = right_orig_img.shape
    resize_factor = round(300/max(img_shape),3) # resize max(length, width) to 300 pixels
    right_orig_img_resize = cv.resize(right_orig_img, None, fx = resize_factor , fy = resize_factor, interpolation = cv.INTER_AREA)
    
    color_contours = (0, 255, 0) # green - color for contours
    
    ### right hand
    bw_img_fuse = getBinaryImage(right_img_fuse)
    
    # resize the right hand mask to match that of the original image
    img_shape = right_orig_img.shape[0:2]
    bw_img = cv.resize(bw_img_fuse, (img_shape[1], img_shape[0]), interpolation = cv.INTER_AREA)

    ###
    contours_hull = getContoursAndHull(bw_img)

    ###
    dw_img = drawConvexHull(right_orig_img, contours_hull['contours'], contours_hull['hierarchy'], contours_hull['hull'])

    # apply mediapipe to get hand landmarks
    results = hands.process(cv.cvtColor(right_orig_img_resize, cv.COLOR_BGR2RGB))

    # to draw all landmarks from mediapipe
    if not results.multi_hand_landmarks:
        cv.imwrite(current_right_target_path, dw_img)
        right_dip_rects[orig_files[index]] = {}
        right_fail_index = right_fail_index + 1
    else:
        dw_img = drawConvexHull(right_orig_img, contours_hull['contours'], contours_hull['hierarchy'], contours_hull['hull'], draw_Contour=True)
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(dw_img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # get tip landmarks from results
        tips_landmarks = getTIPLandmarks(results.multi_hand_landmarks, right_orig_img.shape)
        
        # get DIP landmarks from results
        dips_landmarks = getDIPLandmarks(results.multi_hand_landmarks, right_orig_img.shape)
        
        # get PIP landmarks from results
        pips_landmarks = getPIPLandmarks(results.multi_hand_landmarks, right_orig_img.shape)
               
        ## get middle points b/2 dips_landmarks and pips_landmarks
        dip_pip_landmarks = getHalfPoints(dips_landmarks, pips_landmarks)
        
        # get points closest to tips from hull
        tips_hull = getTipsFromHull(tips_landmarks, contours_hull['hull'])
        
        ## get middle points b/w tips_hull and dips_landmarks
        tip_dip_landmarks = getHalfPoints(tips_landmarks, dips_landmarks)

        
        # get subcontours for each finger
        subContours = {}
        
        # get minimum bounding rectangle(min area) for each subContour
        subContourRects = {}
        subContourBoxes = {}
        
        for finger_key in tips_hull:
            subContours[finger_key] = getContourSubsectionJoint(contours_hull['contours'], tips_hull[finger_key],tip_dip_landmarks[finger_key], dip_pip_landmarks[finger_key])
#             subContours[finger_key] = getContourSubsectionPercentage(contours_hull['contours'], tips_hull[finger_key])
            rect = cv.minAreaRect(subContours[finger_key][0])
            box = cv.boxPoints(rect)
            box = np.int0(box)
            subContourRects[finger_key] = rect
            subContourBoxes[finger_key] = [box]
    
         # draw SubContours finger
        for finger_key in subContours:
            cv.drawContours(dw_img, subContours[finger_key], -1, (255,0,0), 1, 8, contours_hull['hierarchy'])
            cv.drawContours(dw_img, subContourBoxes[finger_key],-1,(0,0,255))
            
            # rotated via rects
            cropped_finger = cropWarpedRect(right_orig_img_clone, subContourRects[finger_key])
            curr_file_name = 'right_' + finger_key + '_' + fuse_files[index]
            curr_path = current_right_dip_target_path + '/' + curr_file_name
            if min(cropped_finger.shape) >0 :
                cv.imwrite(curr_path, cropped_finger)
                # upload this corrected cropped finger to synapse via customSynpaseUpload
                customSynapseUpload(curr_path, curr_file_name)

            # unrotated fingers
            cropped_finger_unrotated = cropImageFromContour(right_orig_img_clone, subContourBoxes[finger_key][0])
            curr_file_name = 'right_' + finger_key + '_' + fuse_files[index]
            curr_path = current_right_dip_target_path_unrotated + '/' + curr_file_name
            if min(cropped_finger_unrotated.shape) >0 :
                cv.imwrite(curr_path, cropped_finger_unrotated)
            
        cv.imwrite(current_right_target_path, dw_img)
        right_dip_rects[orig_files[index]] = subContourRects
        right_pass_index = right_pass_index + 1
    
print('right fail percent')
print(100*right_fail_index/(right_fail_index+right_pass_index))



hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_032_handImaging_2020-12-02_20.49.05.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_008_handImaging_2020-03-04_15.43.41.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE2_020_handImaging_2020-12-09_20.27.28.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_036_handImaging_2021-03-24_14.43.51.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SIT

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_003_handImaging_2020-02-21_15.36.43.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_018_handImaging_2020-09-02_17.45.39.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE2_009_handImaging_2020-11-04_18.38.43.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE1_022_handImaging_2020-09-10_20.26.44.jpg___fuse.png

##################################################
 Uploading file to Synapse storage


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################

hand_segmentation_psorcast Jun 16 2021 08_51 Dan//SITE2_021_handImaging_2020-12-09_21.17.48.jpg___fuse.png

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 

In [17]:
# Minimum bounding rects of dips into a dataframe (left hand)
aa_left = pd.DataFrame.from_dict(left_dip_rects,orient = 'index')
aa_left.columns = 'left_' + aa_left.columns
aa_left['image'] = aa_left.index
aa_left.index = range(len(aa_left.index))

# Minimum bounding rects of dips into a dataframe (right hand)
aa_right = pd.DataFrame.from_dict(right_dip_rects,orient = 'index')
aa_right.columns = 'right_' + aa_right.columns
aa_right['image'] = aa_right.index
aa_right.index = range(len(aa_right.index))

# Merge rects from left and right hands
aa = pd.merge(aa_left, aa_right, on = 'image', how = 'outer')
aa.head()

,left_index,left_middle,left_ring,left_pinky,image,right_index,right_middle,right_ring,right_pinky
0,"((1274.9739990234375, 563.9075317382812), (264...","((1223.30126953125, 303.606689453125), (377.81...","((925.3073120117188, 380.61212158203125), (232...","((908.2859497070312, 747.4181518554688), (275....",SITE1_032_handImaging_2020-12-02_20.49.05.jpg,"((827.5348510742188, 608.640380859375), (315.3...","((1330.4083251953125, 431.5927734375), (298.02...","((1687.5947265625, 514.691650390625), (296.851...","((2243.303466796875, 912.344970703125), (271.4..."
1,"((1959.096435546875, 959.32666015625), (333.46...","((1366.1300048828125, 871.173828125), (328.658...","((1013.6923828125, 989.4841918945312), (316.33...","((572.3953857421875, 1292.8841552734375), (292...",SITE1_008_handImaging_2020-03-04_15.43.41.jpg,"((1013.7578735351562, 1193.888427734375), (303...","((1507.9998779296875, 1049.5), (289.9999389648...","((1896.026123046875, 1140.9156494140625), (263...","((2396.1806640625, 1575.949951171875), (252.89..."
2,"((1697.5203857421875, 626.2720336914062), (280...","((1221.2347412109375, 476.596435546875), (277....","((752.6467895507812, 561.343994140625), (275.8...","((298.54876708984375, 858.1840209960938), (271...",SITE1_038_handImaging_2021-03-25_15.27.53.jpg,NaN,NaN,NaN,NaN
3,"((2089.395751953125, 2109.112060546875), (349....","((1532.0, 1870.0), (394.0, 334.0), 90.0)","((1106.4927978515625, 1997.43310546875), (335....","((683.1912841796875, 2356.884033203125), (312....",SITE2_025_handImaging_2021-01-06_21.22.13.jpg,"((1103.2406005859375, 2023.876220703125), (459...","((1695.177001953125, 1875.8438720703125), (354...","((2165.374755859375, 2070.23046875), (356.0566...","((2468.0, 2470.0), (224.0, 282.0), 90.0)"
4,"((1680.80322265625, 707.2019653320312), (214.9...","((1219.3660888671875, 619.9638061523438), (201...","((908.4500732421875, 668.689453125), (349.4543...","((419.800048828125, 1085.10009765625), (201.24...",SITE1_035_handImaging_2021-02-22_19.17.04.jpg,"((745.09619140625, 842.1519775390625), (248.40...","((1204.5865478515625, 673.6071166992188), (226...","((1563.36474609375, 737.1851806640625), (232.5...","((2065.315185546875, 1100.430419921875), (236...."


In [16]:
### Upload nail bounding rects to Synapse

## Write to csv
aa.to_csv('dip_bounding_rects.csv')

# Synapse target folder
syn_target_folder = 'syn22342373'

# Upload results to Synapse

# provenance
provenance_set = synapseclient.Activity(
    name='dip_bounding_rects.csv',
    description='Minimum bounding rectangle for dips (except thumb). Each rectangle is of the form (center, size, theta)',
    used = 'syn25999658',
    executed = 'https://github.com/itismeghasyam/psorcastValidationAnalysis/blob/master/feature_extraction/dip_segmentation_mediapipe_pipeline.ipynb')

test_entity = synapseclient.File(
    'nail_bounding_rects.csv',
    description='Minimum bounding rectangle for dips (except thumb). Each rectangle is of the form (center, size, theta)',
    parent=syn_target_folder)

test_entity = syn.store(test_entity, activity = provenance_set)



##################################################
 Uploading file to Synapse storage 
##################################################

